# Europe

### Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from ipywidgets import widgets
import numpy as np 
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

## Key indicators

**Source: [https://digital-agenda-data.eu/](https://digital-agenda-data.eu/)**

### Key indicators used:
1) Broadband take-up and coverage
- Standard fixed broadband coverage/availability (bb_scov)
- NGA broadband coverage/availability (bb_ngacov)
- Households with a broadband connection (h_broad)
- Households with fixed broadband connection (h_bbfix)

2) Broadband speeds and prices
- Share of fixed broadband subscriptions >= 100 Mbps (bb_speed100)
- Households with no access to Internet at home, because the costs are too high (all households) (h_xcost)

3) Mobile market
- 4G mobile broadband coverage (total) (mbb_ltecov)

4) Internet usage
- Households with access to the Internet at home (h_iacc)
- Individuals who have never used the internet (all individuals) (i_iux)

5) Take up of internet services (all individuals)
- Using online banking (i_iubk)
- Uploading self-created content to be shared (i_iuupl)
- Participating in social networks, over the internet, last 3 months (i_iusnet)
- Looking for a job or sending a job application (i_iujob)
- Doing an online course (i_iuolc)
- Taking part in on-line consultations or voting to define civic or political issues (i_iuvote)

6) eGovernment
- Individuals interacting online with public authorities, last 12 months (i_iugov12)

7) Digital skills
- Individuals with at least basic digital skills (i_dsk_bab)
- Science and technology graduates (st_grad)

8) ICT Specialists
- Individuals who have obtained ICT skills through formal educational institutions (i_skedu)

9) Security and privacy
- Individuals caught a virus or other computer infection resulting in loss of information or time (i_secvir1)
- Security concerns kept individual from ordering or buying goods or services for private use (i_sbgood)

### Preprocessing

In [2]:
key_indicators = pd.read_csv("data/digital-agenda-scoreboard-key-indicators.csv")
# Drop useless columns
key_indicators = key_indicators.drop(['observation', 'flag', 'note'], axis=1)
# Filter out some indicators
my_indicators = ['bb_scov', 'bb_ngacov', 'h_broad', 'h_bbfix', 'bb_speed100', 'h_xcost', 'mbb_ltecov', 'h_iacc', 'i_iux', 'i_iubk', 'i_iuupl', 'i_iusnet', 'i_iujob', 'i_iuolc', 'i_iuvote', 'i_iugov12', 'i_dsk_s_bab', 'i_skedu', 'i_secvir1', 'i_sbgood']
key_indicators = key_indicators[key_indicators['indicator'].isin(my_indicators)]
# Map the indicators' codes to the corresponding name
indicators_mapping = {'bb_scov' : 'Standard fixed broadband coverage/availability',
           "bb_ngacov" : 'NGA broadband coverage/availability',
           'h_broad' : 'Households with a broadband connection',
           'h_bbfix' : 'Households with fixed broadband connection',
           'bb_speed100' : 'Share of fixed broadband subscriptions >= 100 Mbps',
           'h_xcost' : 'Households with no access to Internet at home, because the costs are too high',
           'mbb_ltecov' : '4G mobile broadband coverage',
           'h_iacc' : 'Households with access to the Internet at home',
           'i_iux' : 'Individuals who have never used the internet (all individuals)',
           'i_iubk' : 'Using online banking',
           'i_iuupl' : 'Uploading self-created content to be shared',
           'i_iusnet' : 'Participating in social networks, over the internet, last 3 months',
           'i_iujob' : 'Looking for a job or sending a job application',
           'i_iuolc' : 'Doing an online course',
           'i_iuvote' : 'Taking part in on-line consultations or voting to define civic or political issues',
           'i_iugov12' : 'Individuals interacting online with public authorities, last 12 months',
           'i_dsk_s_bab' : 'Individuals with at least basic digital skills in Software domain',
           'i_skedu' : 'Individuals who have obtained ICT skills through formal educational institutions',
           'i_secvir1' : 'Individuals caught a virus or other computer infection resulting in loss of information or time',
           'i_sbgood' : 'Security concerns kept individual from ordering or buying goods or services for private use',
          }
key_indicators['indicator'] = key_indicators['indicator'].replace(indicators_mapping)

    
# Mapping between 'ref_area' and the name of the country
countries_mapping =  { 'EE' : 'Estonia',
                        'EL' : 'Greece',
                        'ES' : 'Spain',
                        'FI' : 'Finland',
                        'FR' : 'France',
                        'HR' : 'Croatia',
                        'HU' : 'Hungary',
                        'IE' : 'Ireland',
                        'IS' : 'Iceland',
                        'DE' : 'Deutschland',
                        'CZ' : 'Czech Rep.',
                        'DK' : 'Denmark',
                        'IT' : 'Italy',
                        'LT' : 'Lithuania',
                        'LU' : 'Luxembourg',
                        'LV' : 'Latvia',
                        'MT' : 'Malta',
                        'NL' : 'Netherlands',
                        'NO' : 'Norway',
                        'PT' : 'Portugal',
                        'PL' : 'Poland',
                        'EU' : 'Europe',
                        'RO' : 'Romania',
                        'SE' : 'Sweden',
                        'SI' : 'Slovenia',
                        'SK' : 'Slovakia',
                        'UK' : 'United Kingdom',
                        'AT' : 'Austria',
                        'BE' : 'Belgium',
                        'BG' : 'Bulgaria',
                        'CY' : 'Cyprus',
                        'CH' : 'Switzerland'
                     }
key_indicators['ref_area'] = key_indicators['ref_area'].replace(countries_mapping)

# Drop the index column
key_indicators.reset_index(drop=True, inplace=True)

# In some cases the types in 'time_period' are mixed ==> cast all to strings 
key_indicators['time_period'] = key_indicators['time_period'].apply(lambda j: str(j))
key_indicators['time_period'] = key_indicators['time_period'].apply(lambda j: j[:4])
def convert_to_date(j):
   if len(j.split('-')) == 1:
      return str(datetime.strptime(j, '%Y'))
   elif len(j.split('-')) == 2:
      return str(datetime.strptime(j, '%Y-%m'))
   else:
      return str(datetime.strptime(j, '%Y-%m-%d'))
# key_indicators['time_period'] = key_indicators['time_period'].apply(lambda j: convert_to_date(j))

C:\Users\Roncax\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
 labels = list(key_indicators['indicator'].unique())

### Plot

In [4]:
#key_indicators['indicator'].unique()
key_indicators['time_period'].unique()

array(['2019', '2011', '2012', '2013', '2014', '2017', '2018', '2015',
       '2016', '2010', '2008', '2009', '2007', '2005', '2006', '2004',
       '2003', '2002'], dtype=object)

In [5]:
countries_list = [country for country in countries_mapping.values()]

for country in countries_list:
        # Check if there are multiple breakdowns: if so, keep only the 'total' one
        if len(key_indicators['breakdown'].unique()) > 1:
            if len(key_indicators['unit_measure'].unique()) > 1:
                key_indicators = key_indicators.loc[(key_indicators['breakdown'].str.contains('total')) & (key_indicators['unit_measure'] == 'pc_ind')]
            else:
                key_indicators = key_indicators.loc[(data['breakdown'].str.contains('total'))]
                
#key_indicators.loc[key_indicators['time_period'] == '2019'].to_csv('indicators.csv')

In [6]:
year1 = widgets.IntSlider(
    value=2019,
    min=2002,
    max=2019,
    step=1,
    description='Year:',
    continuous_update=True
)

year  = widgets.Dropdown(
    options=list(key_indicators['time_period'].unique()),
    value='2019',
    description='Years: ',
)


container = widgets.HBox([year])

indicators_menu = widgets.Dropdown(
    options=list(key_indicators['indicator'].unique()),
    value='Uploading self-created content to be shared',
    description='Indicators: ',
)

trace1 = go.Bar(x=key_indicators['ref_area'], y = key_indicators['value'], opacity=0.75, textposition='auto')
layout=go.Layout(
                        title=dict(
                            text='Europe digital divide key indicators'
                        ),
                        barmode='overlay',
                        xaxis={'categoryorder':'total descending'}
                    )
g = go.FigureWidget(data=trace1,
                    layout = layout
                    )

In [7]:
def response(change):
    filter_list = [i and j for i, j in
                    zip(key_indicators['time_period'] == year.value,
                    key_indicators['indicator'] == indicators_menu.value)]
    temp_df = key_indicators[filter_list]
    x1 = temp_df['ref_area']
    y1 = temp_df['value']
    #x2 = temp_df['dep_delay']
    with g.batch_update():
        g.data[0].x = x1
        g.data[0].y = y1
        #g.data[1].x = x2
        g.layout.barmode = 'overlay'
        g.layout.xaxis.title = 'Area'
        g.layout.yaxis.title = 'Percentage'


indicators_menu.observe(response, names="value")
year.observe(response, names="value")

In [8]:
container2 = widgets.HBox([indicators_menu])
widgets.VBox([container,
              container2,
              g])

## DESI

**Source: [https://digital-agenda-data.eu/datasets/desi/](https://digital-agenda-data.eu/datasets/desi/)**

### DESI indicators used:
1) Broadband take-up and coverage
- Standard fixed broadband coverage/availability (bb_scov)
- NGA broadband coverage/availability (bb_ngacov)
- Households with a broadband connection (h_broad)
- Households with fixed broadband connection (h_bbfix)

2) Broadband speeds and prices
- Share of fixed broadband subscriptions >= 100 Mbps (bb_speed100)
- Households with no access to Internet at home, because the costs are too high (all households) (h_xcost)

3) Mobile market
- 4G mobile broadband coverage (total) (mbb_ltecov)

4) Internet usage
- Households with access to the Internet at home (h_iacc)
- Individuals who have never used the internet (all individuals) (i_iux)

5) Take up of internet services (all individuals)
- Using online banking (i_iubk)
- Uploading self-created content to be shared (i_iuupl)
- Participating in social networks, over the internet, last 3 months (i_iusnet)
- Looking for a job or sending a job application (i_iujob)
- Doing an online course (i_iuolc)
- Taking part in on-line consultations or voting to define civic or political issues (i_iuvote)

6) eGovernment
- Individuals interacting online with public authorities, last 12 months (i_iugov12)

7) Digital skills
- Individuals with at least basic digital skills (i_dsk_bab)
- Science and technology graduates (st_grad)

8) ICT Specialists
- Individuals who have obtained ICT skills through formal educational institutions (i_skedu)

9) Security and privacy
- Individuals caught a virus or other computer infection resulting in loss of information or time (i_secvir1)
- Security concerns kept individual from ordering or buying goods or services for private use (i_sbgood)

In [ ]:
desi = pd.read_csv("data/digital-agenda-scoreboard-key-indicators.csv")
# Drop useless columns
desi = desi.drop(['observation', 'flag', 'note'], axis=1)
# Filter out some indicators
my_indicators = ['bb_scov', 'bb_ngacov', 'h_broad', 'h_bbfix', 'bb_speed100', 'h_xcost', 'mbb_ltecov', 'h_iacc', 'i_iux', 'i_iubk', 'i_iuupl', 'i_iusnet', 'i_iujob', 'i_iuolc', 'i_iuvote', 'i_iugov12', 'i_dsk_s_bab', 'i_skedu', 'i_secvir1', 'i_sbgood']
desi = desi[key_indicators['indicator'].isin(my_indicators)]
# Map the indicators' codes to the corresponding name
indicators_mapping = {'bb_scov' : 'Standard fixed broadband coverage/availability',
           "bb_ngacov" : 'NGA broadband coverage/availability',
           'h_broad' : 'Households with a broadband connection',
           'h_bbfix' : 'Households with fixed broadband connection',
           'bb_speed100' : 'Share of fixed broadband subscriptions >= 100 Mbps',
           'h_xcost' : 'Households with no access to Internet at home, because the costs are too high',
           'mbb_ltecov' : '4G mobile broadband coverage',
           'h_iacc' : 'Households with access to the Internet at home',
           'i_iux' : 'Individuals who have never used the internet (all individuals)',
           'i_iubk' : 'Using online banking',
           'i_iuupl' : 'Uploading self-created content to be shared',
           'i_iusnet' : 'Participating in social networks, over the internet, last 3 months',
           'i_iujob' : 'Looking for a job or sending a job application',
           'i_iuolc' : 'Doing an online course',
           'i_iuvote' : 'Taking part in on-line consultations or voting to define civic or political issues',
           'i_iugov12' : 'Individuals interacting online with public authorities, last 12 months',
           'i_dsk_s_bab' : 'Individuals with at least basic digital skills in Software domain',
           'i_skedu' : 'Individuals who have obtained ICT skills through formal educational institutions',
           'i_secvir1' : 'Individuals caught a virus or other computer infection resulting in loss of information or time',
           'i_sbgood' : 'Security concerns kept individual from ordering or buying goods or services for private use',
          }
desi['indicator'] = desi['indicator'].replace(indicators_mapping)

    
# Mapping between 'ref_area' and the name of the country
countries_mapping =  { 'EE' : 'Estonia',
                        'EL' : 'Greece',
                        'ES' : 'Spain',
                        'FI' : 'Finland',
                        'FR' : 'France',
                        'HR' : 'Croatia',
                        'HU' : 'Hungary',
                        'IE' : 'Ireland',
                        'IS' : 'Iceland',
                        'DE' : 'Deutschland',
                        'CZ' : 'Czech Rep.',
                        'DK' : 'Denmark',
                        'IT' : 'Italy',
                        'LT' : 'Lithuania',
                        'LU' : 'Luxembourg',
                        'LV' : 'Latvia',
                        'MT' : 'Malta',
                        'NL' : 'Netherlands',
                        'NO' : 'Norway',
                        'PT' : 'Portugal',
                        'PL' : 'Poland',
                        'EU' : 'Europe',
                        'RO' : 'Romania',
                        'SE' : 'Sweden',
                        'SI' : 'Slovenia',
                        'SK' : 'Slovakia',
                        'UK' : 'United Kingdom',
                        'AT' : 'Austria',
                        'BE' : 'Belgium',
                        'BG' : 'Bulgaria',
                        'CY' : 'Cyprus',
                        'CH' : 'Switzerland'
                     }
desi['ref_area'] = desi['ref_area'].replace(countries_mapping)

# Drop the index column
desi.reset_index(drop=True, inplace=True)

# In some cases the types in 'time_period' are mixed ==> cast all to strings 
desi['time_period'] = desi['time_period'].apply(lambda j: str(j))
desi['time_period'] = desi['time_period'].apply(lambda j: j[:4])
def convert_to_date(j):
   if len(j.split('-')) == 1:
      return str(datetime.strptime(j, '%Y'))
   elif len(j.split('-')) == 2:
      return str(datetime.strptime(j, '%Y-%m'))
   else:
      return str(datetime.strptime(j, '%Y-%m-%d'))
# desi['time_period'] = desi['time_period'].apply(lambda j: convert_to_date(j))

In [ ]:
fig.write_html("barchart.html")